SetUp Paths


In [3]:
Workspace_path = "./Tensorflow/workspace"
scripts_path = "./Tensorflow/scripts"
annotations_path = Workspace_path + "/annotations"
api_model_path = "C:/Users/Asus/models"
image_path = Workspace_path + "/images"
model_path = Workspace_path + "/models"
pretrainedModel_path = Workspace_path + "/pre-trained-models"
# config_path
checkpoint_path = model_path + "/my_ssd_mobilenet"

Labels

In [4]:
labels = [
    {"name":"with_mask", "id":1},
    {"name":"without_mask", "id":2},
    {"name":"mask_weared_incorrect", "id":3},
]

Create Label Map for Tensorflow

In [5]:
with open(annotations_path + "/labelMap.pbtxt", "w") as f:
    for label in labels:
        f.write("item{\n")
        f.write(f"\tname:'{label['name']}'\n")
        f.write(f"\tid:{label['id']}\n")
        f.write("}\n")

Creating Tensorflow records

In [6]:
!python {scripts_path + "/generate_tfrecord.py"} -x {image_path + "/train"} -l {annotations_path + "/labelMap.pbtxt"} -o {annotations_path + "/train.record"}
!python {scripts_path + "/generate_tfrecord.py"} -x {image_path + "/test"} -l {annotations_path + "/labelMap.pbtxt"} -o {annotations_path + "/test.record"}

Successfully created the TFRecord file: ./Tensorflow/workspace/annotations/train.record
Successfully created the TFRecord file: ./Tensorflow/workspace/annotations/test.record


Copy model Config to Training Folder

In [6]:
#Copy only the config file from the pretrained models into the 
#models folder

Import necessary libraries

In [7]:
import tensorflow as tf
from object_detection.utils import config_util
from object_detection.protos import pipeline_pb2
from google.protobuf import text_format

In [8]:
config_path = model_path + "/my_ssd_mobilenet/pipeline.config"
config = config_util.get_configs_from_pipeline_file(config_path)

In [9]:
config

{'model': ssd {
   num_classes: 3
   image_resizer {
     fixed_shape_resizer {
       height: 320
       width: 320
     }
   }
   feature_extractor {
     type: "ssd_mobilenet_v2_fpn_keras"
     depth_multiplier: 1.0
     min_depth: 16
     conv_hyperparams {
       regularizer {
         l2_regularizer {
           weight: 3.9999998989515007e-05
         }
       }
       initializer {
         random_normal_initializer {
           mean: 0.0
           stddev: 0.009999999776482582
         }
       }
       activation: RELU_6
       batch_norm {
         decay: 0.996999979019165
         scale: true
         epsilon: 0.0010000000474974513
       }
     }
     use_depthwise: true
     override_base_feature_extractor_hyperparams: true
     fpn {
       min_level: 3
       max_level: 7
       additional_layer_depth: 128
     }
   }
   box_coder {
     faster_rcnn_box_coder {
       y_scale: 10.0
       x_scale: 10.0
       height_scale: 5.0
       width_scale: 5.0
     }
   }
   match

In [10]:
pipeline_config = pipeline_pb2.TrainEvalPipelineConfig()
with tf.io.gfile.GFile(config_path, "r") as f:
    proto_str = f.read()
    text_format.Merge(proto_str, pipeline_config)

In [11]:
pipeline_config

model {
  ssd {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

In [12]:
pipeline_config.model.ssd.num_classes = 3
pipeline_config.train_config.batch_size = 4
pipeline_config.train_config.fine_tune_checkpoint = checkpoint_path + "/ckpt-0"
pipeline_config.train_config.fine_tune_checkpoint_type = "detection"
pipeline_config.train_input_reader.label_map_path = annotations_path + "/labelMap.pbtxt"
pipeline_config.train_input_reader.tf_record_input_reader.input_path[:] = [annotations_path + "/train.record"]
# pipeline_config.eval_input_reader.label_map_path = annotations_path + "/labelMap.pbtxt"
# pipeline_config.eval_input_reader.tf_record_input_reader.input_path[:] = [annotations_path + "/test.record"]


In [13]:
pipeline_config

model {
  ssd {
    num_classes: 3
    image_resizer {
      fixed_shape_resizer {
        height: 320
        width: 320
      }
    }
    feature_extractor {
      type: "ssd_mobilenet_v2_fpn_keras"
      depth_multiplier: 1.0
      min_depth: 16
      conv_hyperparams {
        regularizer {
          l2_regularizer {
            weight: 3.9999998989515007e-05
          }
        }
        initializer {
          random_normal_initializer {
            mean: 0.0
            stddev: 0.009999999776482582
          }
        }
        activation: RELU_6
        batch_norm {
          decay: 0.996999979019165
          scale: true
          epsilon: 0.0010000000474974513
        }
      }
      use_depthwise: true
      override_base_feature_extractor_hyperparams: true
      fpn {
        min_level: 3
        max_level: 7
        additional_layer_depth: 128
      }
    }
    box_coder {
      faster_rcnn_box_coder {
        y_scale: 10.0
        x_scale: 10.0
        height_scale: 5.0
 

In [14]:
config_text = text_format.MessageToString(pipeline_config)
with tf.io.gfile.GFile(config_path, "wb") as f:
    f.write(config_text)
# add the 2 data changes which are commented earlier manually in the file

Train the MobileNet

In [15]:
#run the model_main_tf2.py with specified flags in cmd
#python C:/Users/Asus/models/research/object_detection/model_main_tf2.py --model_dir=Tensorflow/workspace/models/my_ssd_mobilenet --pipeline_config_path=Tensorflow/workspace/models/my_ssd_mobilenet/pipeline.config --num_train_steps=5000

Load our trained Model

In [16]:
import os
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils
from object_detection.builders import model_builder

In [17]:
configs = config_util.get_configs_from_pipeline_file(config_path)
detectionModel = model_builder.build(model_config=configs['model'], is_training=False)

In [18]:
checkpoint = tf.compat.v2.train.Checkpoint(model=detectionModel)
checkpoint.restore(os.path.join(checkpoint_path, "ckpt-4")).expect_partial()

In [19]:
def detectImage(image):
    image, shape = detectionModel.preprocess(image)
    predictDict = detectionModel.predict(image, shape)
    detections = detectionModel.postprocess(predictDict, shape)
    return detections

Detect in Real Time

In [20]:
import cv2
import numpy as np

In [21]:
category_index = label_map_util.create_category_index_from_labelmap(annotations_path + "/labelMap.pbtxt")

In [22]:
category_index

{1: {'id': 1, 'name': 'with_mask'},
 2: {'id': 2, 'name': 'without_mask'},
 3: {'id': 3, 'name': 'mask_weared_incorrect'}}

In [23]:
capture = cv2.VideoCapture(0)
width = int(capture.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

In [24]:
while True:
    ret, frame = capture.read()
    image_np = np.array(frame)

    input_tensor = tf.convert_to_tensor(np.expand_dims(image_np, 0), dtype=tf.float32)
    detections = detectImage(input_tensor)

    nums_detections = int(detections.pop("num_detections"))

    detections = {key:value[0, :nums_detections].numpy() for key, value in detections.items()}
    detections["num_detections"] = nums_detections
    detections["detection_classes"] = detections["detection_classes"].astype(np.int64)

    label_id_offset = 1
    image_np_with_detections = image_np.copy()

    visualization_utils.visualize_boxes_and_labels_on_image_array(
        image_np_with_detections,
        detections['detection_boxes'],
        detections['detection_classes'] + label_id_offset,
        detections['detection_scores'],
        category_index,
        use_normalized_coordinates=True,
        max_boxes_to_draw=1,
        min_score_thresh=0.3,
        agnostic_mode=False
    )

    cv2.imshow("object_detection", cv2.resize(image_np_with_detections, (800, 600)))

    if cv2.waitKey(1) & 0xFF == ord("q"):
        capture.releasqe()
        break

: 